In [1]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [3]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [4]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [5]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [16]:
crdf = complete_reports_df
cmdf = complete_measurements_df

categories = list(crdf.category.sort_values().unique())
selected_year = 2017

selected_vars = ['v17', 'v18']
default_cat = 'A'

ydf = crdf[crdf.year == selected_year]

vdf = variables_df
v1_name = vdf[vdf.var_id == int(selected_vars[0][1:])].name.iloc[0]
v1_unit = vdf[vdf.var_id == int(selected_vars[0][1:])].unit.iloc[0]
v2_name = vdf[vdf.var_id == int(selected_vars[1][1:])].name.iloc[0]
v2_unit = vdf[vdf.var_id == int(selected_vars[1][1:])].unit.iloc[0]

data = []

for i, category in enumerate(categories):
    fdf = ydf[ydf.category == category].fillna(0)
    total_col = fdf[selected_vars[0]] + fdf[selected_vars[1]]
    fdf = fdf.assign(total=total_col).sort_values('total')
    visible = True if i == 0 else False

    data.append(go.Bar(
        x=list(fdf.code),
        y=list(fdf[selected_vars[0]]),
        text= [f'{str(val)[:-2]}' for val in list(fdf[selected_vars[0]])],
        opacity=0.8,  
        textposition='auto',
        hoverinfo='text',
        visible=visible,
        name='Agua Potable',
    ))
    
    data.append(go.Bar(
        x=list(fdf.code),
        y=list(fdf[selected_vars[1]]),
        text= [ f'{str(val)[:-2]}' for val in list(fdf[selected_vars[1]])],
        opacity=0.8,  
        textposition='auto',
        hoverinfo='text',
        visible=visible,
        name='Alcantarillado',
    ))

def get_visible_list(category):
    category_map = dict(A=0, B=1, C=2, D=3)
    base_array = [False for i in range(4)]
    base_array[category_map[category]] = True
    fat_array = [[x] * 2 for x in base_array]
    return [x for l in fat_array for x in l]

def make_x_axis(category):
    return dict(
        title=f'EPSAs Categoría {category}',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

def make_y_axis():
    return dict(
        title=f'Numero de Conexiones',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

updatemenus = [dict(
    type= 'buttons',
    showactive=True,
    active=0,
    xanchor='left',
    yanchor='top',
    direction='right',
    x= 0.25,
    y= -0.3,
    buttons = [dict(
        label= f'Categoría {cat}',
        method='update',
        args = [
            {'visible': get_visible_list(cat)},
            dict(
                title= f'Presión del Servicio de Agua Potable - Categoría {cat}',
                xaxis= make_x_axis(cat),
            ),
        ],
    ) for cat in categories]
)]

layout = go.Layout(
    title= 'Conexiones de Agua Potable y Alcantarillado',
    xaxis= make_x_axis('A'),
    yaxis= make_y_axis(),
    hovermode='closest',
    updatemenus=updatemenus,
    barmode='stack',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [103]:
import plotly.plotly as py
py.iplot(fig, filename='connections_2017')